In [ ]:
# This is training the breakfast model, but using dnn instead of linear regressor

In [2]:
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

1.10.0


In [3]:
CSV_COLUMNS = ['num_sandwitch', 'num_salad','num_coke','total_cost']
LABEL_COLUMN = 'total_cost'
DEFAULTS = [[0], [0], [0], [0]]

def read_dataset(filename, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(value_column, record_defaults= DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label if mode != tf.estimator.ModeKeys.PREDICT else features
        
        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        filenames_dataset = tf.data.Dataset.list_files(filename)
        # Read lines from text files
        textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = textlines_dataset.map(decode_csv)
        
        if mode == tf.estimator.ModeKeys.TRAIN:        # if training
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:        # if evaluating or validating, epochs = 1 since it doesn't have to repeat
            num_epochs = 1 # end-of-input after this
        
        # make the dataset repeat as the number of epochs, and then batch them to specific batch size
        dataset = dataset.repeat(num_epochs).batch(batch_size)
    
        return dataset.make_one_shot_iterator().get_next()        # # Creates an Iterator for enumerating the elements of this dataset
    return _input_fn

In [4]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('num_sandwitch'),
    tf.feature_column.numeric_column('num_salad'),
    tf.feature_column.numeric_column('num_coke'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

In [5]:
# Serving Input Function
# Defines the expected shape of the JSON feed that the model
# will receive once deployed behind a REST API in production.
def serving_input_fn():
  feature_placeholders = {
    'num_sandwitch' : tf.placeholder(tf.int32, shape=[None]),
    'num_salad' : tf.placeholder(tf.int32, shape=[None]),
    'num_coke' : tf.placeholder(tf.int32, shape=[None]),
  }
  # You can transforma data here from the input format to the format expected by your model.
  features = feature_placeholders # no transformation needed
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)  # input_fn for training and for commercial use

In [28]:
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNRegressor(
        model_dir=output_dir,
        feature_columns=feature_cols,
        hidden_units=[32,16,8],
        optimizer=tf.train.AdamOptimizer(
            learning_rate=0.1,
        )
    )

    # train specs are parameters to run training
    train_spec=tf.estimator.TrainSpec(
                       input_fn = read_dataset('./data/breakfast_train*.csv', mode = tf.estimator.ModeKeys.TRAIN),
                       max_steps = num_train_steps)        # use max_steps here since epochs is infinite
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)

    # eval specs are parameters to run evaluation
    eval_spec=tf.estimator.EvalSpec(
                       input_fn = read_dataset('./data/breakfast_valid*.csv', mode = tf.estimator.ModeKeys.EVAL),
                       steps = None,        # no stepping because evaluation is for testing accuracy, not training
                       start_delay_secs = 1, # start evaluating after N seconds
                       throttle_secs = 1,  # evaluate every N seconds
                       exporters = exporter)        # eval is designed to be realistic, so exporter instead of training dataset will be used here
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    
    return estimator

In [29]:
OUTDIR = './output_breakfast2'
#TensorBoard().start(OUTDIR)

In [30]:
# Run training    
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
estimator = train_and_evaluate(OUTDIR, num_train_steps = 80000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './output_breakfast2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fec746cd668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling

INFO:tensorflow:global_step/sec: 101.669
INFO:tensorflow:loss = 1888.4548, step = 7101 (0.984 sec)
INFO:tensorflow:global_step/sec: 100.973
INFO:tensorflow:loss = 8130348.0, step = 7201 (0.990 sec)
INFO:tensorflow:global_step/sec: 100.961
INFO:tensorflow:loss = 5490.6216, step = 7301 (0.991 sec)
INFO:tensorflow:global_step/sec: 101.17
INFO:tensorflow:loss = 395183.25, step = 7401 (0.988 sec)
INFO:tensorflow:global_step/sec: 101.017
INFO:tensorflow:loss = 170197.97, step = 7501 (0.990 sec)
INFO:tensorflow:global_step/sec: 100.621
INFO:tensorflow:loss = 445.93518, step = 7601 (0.994 sec)
INFO:tensorflow:global_step/sec: 100.33
INFO:tensorflow:loss = 53300.66, step = 7701 (0.997 sec)
INFO:tensorflow:global_step/sec: 102.775
INFO:tensorflow:loss = 15309.847, step = 7801 (0.973 sec)
INFO:tensorflow:global_step/sec: 101.285
INFO:tensorflow:loss = 368.92987, step = 7901 (0.987 sec)
INFO:tensorflow:global_step/sec: 100.693
INFO:tensorflow:loss = 1028.1499, step = 8001 (0.993 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 99.916
INFO:tensorflow:loss = 166468.66, step = 15401 (1.001 sec)
INFO:tensorflow:global_step/sec: 100.317
INFO:tensorflow:loss = 2171.783, step = 15501 (0.997 sec)
INFO:tensorflow:global_step/sec: 101.371
INFO:tensorflow:loss = 1667.9858, step = 15601 (0.986 sec)
INFO:tensorflow:global_step/sec: 108.865
INFO:tensorflow:loss = 14523972.0, step = 15701 (0.919 sec)
INFO:tensorflow:global_step/sec: 95.9262
INFO:tensorflow:loss = 1386.5483, step = 15801 (1.042 sec)
INFO:tensorflow:global_step/sec: 100.85
INFO:tensorflow:loss = 237.98282, step = 15901 (0.992 sec)
INFO:tensorflow:global_step/sec: 101.577
INFO:tensorflow:loss = 1678.3696, step = 16001 (0.984 sec)
INFO:tensorflow:global_step/sec: 99.7582
INFO:tensorflow:loss = 6047.248, step = 16101 (1.002 sec)
INFO:tensorflow:global_step/sec: 100.655
INFO:tensorflow:loss = 119.80018, step = 16201 (0.993 sec)
INFO:tensorflow:global_step/sec: 101.238
INFO:tensorflow:loss = 560.2759, step = 16301 (0.988 sec)
INFO

INFO:tensorflow:loss = 124.673355, step = 23601 (1.537 sec)
INFO:tensorflow:global_step/sec: 66.075
INFO:tensorflow:loss = 11910.903, step = 23701 (1.514 sec)
INFO:tensorflow:global_step/sec: 62.4342
INFO:tensorflow:loss = 213.93645, step = 23801 (1.602 sec)
INFO:tensorflow:global_step/sec: 69.1263
INFO:tensorflow:loss = 16450572.0, step = 23901 (1.447 sec)
INFO:tensorflow:global_step/sec: 64.1206
INFO:tensorflow:loss = 368.9187, step = 24001 (1.559 sec)
INFO:tensorflow:global_step/sec: 64.8212
INFO:tensorflow:loss = 8.078954, step = 24101 (1.543 sec)
INFO:tensorflow:global_step/sec: 66.2279
INFO:tensorflow:loss = 0.7958885, step = 24201 (1.510 sec)
INFO:tensorflow:global_step/sec: 64.6501
INFO:tensorflow:loss = 7696481.5, step = 24301 (1.547 sec)
INFO:tensorflow:global_step/sec: 65.5253
INFO:tensorflow:loss = 18.685957, step = 24401 (1.526 sec)
INFO:tensorflow:global_step/sec: 65.6855
INFO:tensorflow:loss = 5.6456785, step = 24501 (1.523 sec)
INFO:tensorflow:global_step/sec: 62.5644
I

INFO:tensorflow:global_step/sec: 64.4981
INFO:tensorflow:loss = 5.2672024, step = 31901 (1.550 sec)
INFO:tensorflow:global_step/sec: 49.8262
INFO:tensorflow:loss = 377404.9, step = 32001 (2.008 sec)
INFO:tensorflow:global_step/sec: 49.8566
INFO:tensorflow:loss = 21.048637, step = 32101 (2.006 sec)
INFO:tensorflow:global_step/sec: 54.6937
INFO:tensorflow:loss = 2.2514753, step = 32201 (1.828 sec)
INFO:tensorflow:global_step/sec: 55.7265
INFO:tensorflow:loss = 919.78625, step = 32301 (1.797 sec)
INFO:tensorflow:global_step/sec: 45.2698
INFO:tensorflow:loss = 6852.3535, step = 32401 (2.207 sec)
INFO:tensorflow:global_step/sec: 56.2947
INFO:tensorflow:loss = 12.448349, step = 32501 (1.776 sec)
INFO:tensorflow:global_step/sec: 46.6512
INFO:tensorflow:loss = 1.6705055, step = 32601 (2.145 sec)
INFO:tensorflow:global_step/sec: 46.0297
INFO:tensorflow:loss = 0.22554679, step = 32701 (2.172 sec)
INFO:tensorflow:global_step/sec: 37.1654
INFO:tensorflow:loss = 1343115.9, step = 32801 (2.693 sec)


...

INFO:tensorflow:global_step/sec: 51.9447
INFO:tensorflow:loss = 43928.633, step = 46201 (1.923 sec)
INFO:tensorflow:global_step/sec: 63.4664
INFO:tensorflow:loss = 84800.516, step = 46301 (1.575 sec)
INFO:tensorflow:global_step/sec: 41.9419
INFO:tensorflow:loss = 2043.2056, step = 46401 (2.384 sec)
INFO:tensorflow:global_step/sec: 49.044
INFO:tensorflow:loss = 1502740.8, step = 46501 (2.039 sec)
INFO:tensorflow:global_step/sec: 45.3889
INFO:tensorflow:loss = 420.19754, step = 46601 (2.203 sec)
INFO:tensorflow:global_step/sec: 58.8519
INFO:tensorflow:loss = 405187.7, step = 46701 (1.700 sec)
INFO:tensorflow:global_step/sec: 45.4197
INFO:tensorflow:loss = 827183.4, step = 46801 (2.201 sec)
INFO:tensorflow:global_step/sec: 45.76
INFO:tensorflow:loss = 8849.367, step = 46901 (2.185 sec)
INFO:tensorflow:global_step/sec: 56.3942
INFO:tensorflow:loss = 452895.2, step = 47001 (1.773 sec)
INFO:tensorflow:global_step/sec: 53.2291
INFO:tensorflow:loss = 3737.0642, step = 47101 (1.879 sec)
INFO:te

INFO:tensorflow:loss = 45247.85, step = 54401 (1.525 sec)
INFO:tensorflow:global_step/sec: 64.4368
INFO:tensorflow:loss = 63.512165, step = 54501 (1.552 sec)
INFO:tensorflow:global_step/sec: 65.4968
INFO:tensorflow:loss = 1.0513315, step = 54601 (1.528 sec)
INFO:tensorflow:global_step/sec: 63.2702
INFO:tensorflow:loss = 0.1524929, step = 54701 (1.580 sec)
INFO:tensorflow:global_step/sec: 63.6715
INFO:tensorflow:loss = 0.7783323, step = 54801 (1.571 sec)
INFO:tensorflow:global_step/sec: 65.156
INFO:tensorflow:loss = 8379558.5, step = 54901 (1.535 sec)
INFO:tensorflow:global_step/sec: 65.4661
INFO:tensorflow:loss = 1657.6567, step = 55001 (1.527 sec)
INFO:tensorflow:global_step/sec: 66.2055
INFO:tensorflow:loss = 3.2370272, step = 55101 (1.511 sec)
INFO:tensorflow:global_step/sec: 66.5097
INFO:tensorflow:loss = 502607.38, step = 55201 (1.503 sec)
INFO:tensorflow:global_step/sec: 65.0639
INFO:tensorflow:loss = 16.685675, step = 55301 (1.537 sec)
INFO:tensorflow:global_step/sec: 66.1029
IN

INFO:tensorflow:global_step/sec: 65.1389
INFO:tensorflow:loss = 1.9935528, step = 62701 (1.535 sec)
INFO:tensorflow:global_step/sec: 66.1035
INFO:tensorflow:loss = 0.23135397, step = 62801 (1.513 sec)
INFO:tensorflow:global_step/sec: 65.4392
INFO:tensorflow:loss = 20.54074, step = 62901 (1.528 sec)
INFO:tensorflow:global_step/sec: 62.6709
INFO:tensorflow:loss = 5940.3677, step = 63001 (1.596 sec)
INFO:tensorflow:global_step/sec: 63.7685
INFO:tensorflow:loss = 7.437605, step = 63101 (1.567 sec)
INFO:tensorflow:global_step/sec: 64.434
INFO:tensorflow:loss = 1.7586514, step = 63201 (1.552 sec)
INFO:tensorflow:global_step/sec: 66.124
INFO:tensorflow:loss = 0.5711069, step = 63301 (1.513 sec)
INFO:tensorflow:global_step/sec: 65.6414
INFO:tensorflow:loss = 4058255.0, step = 63401 (1.523 sec)
INFO:tensorflow:global_step/sec: 66.0961
INFO:tensorflow:loss = 34.793312, step = 63501 (1.513 sec)
INFO:tensorflow:global_step/sec: 64.908
INFO:tensorflow:loss = 0.30546907, step = 63601 (1.540 sec)
INF

INFO:tensorflow:loss = 6093053.0, step = 70901 (1.044 sec)
INFO:tensorflow:global_step/sec: 101.35
INFO:tensorflow:loss = 204.60333, step = 71001 (0.986 sec)
INFO:tensorflow:global_step/sec: 103.104
INFO:tensorflow:loss = 2.5512538, step = 71101 (0.970 sec)
INFO:tensorflow:global_step/sec: 92.6472
INFO:tensorflow:loss = 0.3197861, step = 71201 (1.079 sec)
INFO:tensorflow:global_step/sec: 93.1542
INFO:tensorflow:loss = 0.030929983, step = 71301 (1.075 sec)
INFO:tensorflow:global_step/sec: 89.6668
INFO:tensorflow:loss = 0.018693134, step = 71401 (1.114 sec)
INFO:tensorflow:global_step/sec: 90.7993
INFO:tensorflow:loss = 18839636.0, step = 71501 (1.101 sec)
INFO:tensorflow:global_step/sec: 101.635
INFO:tensorflow:loss = 867.3726, step = 71601 (0.984 sec)
INFO:tensorflow:global_step/sec: 82.4007
INFO:tensorflow:loss = 7.894717, step = 71701 (1.214 sec)
INFO:tensorflow:global_step/sec: 93.8797
INFO:tensorflow:loss = 1.0059471, step = 71801 (1.065 sec)
INFO:tensorflow:global_step/sec: 104.31

INFO:tensorflow:loss = 288471.2, step = 79101 (1.000 sec)
INFO:tensorflow:global_step/sec: 99.7401
INFO:tensorflow:loss = 20860.176, step = 79201 (1.003 sec)
INFO:tensorflow:global_step/sec: 100.108
INFO:tensorflow:loss = 94930.34, step = 79301 (0.999 sec)
INFO:tensorflow:global_step/sec: 100.835
INFO:tensorflow:loss = 2468.7188, step = 79401 (0.992 sec)
INFO:tensorflow:global_step/sec: 100.947
INFO:tensorflow:loss = 33010.3, step = 79501 (0.991 sec)
INFO:tensorflow:global_step/sec: 95.9113
INFO:tensorflow:loss = 6.351243, step = 79601 (1.043 sec)
INFO:tensorflow:global_step/sec: 97.7594
INFO:tensorflow:loss = 1167021.5, step = 79701 (1.022 sec)
INFO:tensorflow:global_step/sec: 101.283
INFO:tensorflow:loss = 4.86329, step = 79801 (0.987 sec)
INFO:tensorflow:global_step/sec: 100.06
INFO:tensorflow:loss = 0.013581604, step = 79901 (1.000 sec)
INFO:tensorflow:Saving checkpoints for 80000 into ./output_breakfast2/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling mo

In [45]:
def predict_input_fn():
    """An input function for prediction"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices({
            'num_sandwitch' : [10],
            'num_salad' : [10],
            'num_coke' : [10]
        })
    dataset = dataset.batch(1)
    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()  

In [46]:
predictions = estimator.predict(input_fn=predict_input_fn)
print(list(predictions))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./output_breakfast2/model.ckpt-80000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[{'predictions': array([1350.0051], dtype=float32)}]
